In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import random as rn
import seaborn as sns
import matplotlib.pyplot as plt
import time

### Pra proses
from sklearn.preprocessing import MinMaxScaler

### Near Miss under sampling
from imblearn.under_sampling import NearMiss 

### Bagi data
from sklearn.model_selection import train_test_split

### Seleksi fitur
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier

### Long short term memory (LSTM)
from tensorflow.keras.models import Sequential
from tensorflow import keras
from keras.layers import Dense, Dropout, Activation, LSTM, Flatten, Reshape

### metrik pengukuran
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import make_scorer, f1_score

### Tuning Hyperparameter
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

### KerasClassifier wrapper
from keras.wrappers.scikit_learn import KerasClassifier

### save model
import joblib

import warnings
warnings.filterwarnings('ignore')

# menampilkan lebih banyak kolom dan baris pada Dataset
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Tipe Dataset #
# ============ #

tipe_dataset = {'dst_port':'uint32','protocol':'uint8','timestamp':'uint64','flow_duration':'int64','tot_fwd_pkts':'uint32','tot_bwd_pkts':'uint32','totlen_fwd_pkts':'uint32','totlen_bwd_pkts':'uint32','fwd_pkt_len_max':'uint16','fwd_pkt_len_min':'uint16',
'fwd_pkt_len_mean':'float32','fwd_pkt_len_std':'float32','bwd_pkt_len_max':'uint16','bwd_pkt_len_min':'uint16','bwd_pkt_len_mean':'float32','bwd_pkt_len_std':'float32','flow_byts_s':'float64','flow_pkts_s':'float64','flow_iat_mean':'float32','flow_iat_std':'float32',
'flow_iat_max':'int64','flow_iat_min':'int64','fwd_iat_tot':'int64','fwd_iat_mean':'float32','fwd_iat_std':'float32','fwd_iat_max':'int64','fwd_iat_min':'int64','bwd_iat_tot':'uint32','bwd_iat_mean':'float32','bwd_iat_std':'float32',
'bwd_iat_max':'uint32','bwd_iat_min':'uint32','fwd_psh_flags':'uint8','bwd_psh_flags':'uint8','fwd_urg_flags':'uint8','bwd_urg_flags':'uint8','fwd_header_len':'uint32','bwd_header_len':'uint32','fwd_pkts_s':'float32','bwd_pkts_s':'float32',
'pkt_len_min':'uint16','pkt_len_max':'uint16','pkt_len_mean':'float32','pkt_len_std':'float32','pkt_len_var':'float32','fin_flag_cnt':'uint8','syn_flag_cnt':'uint8','rst_flag_cnt':'uint8','psh_flag_cnt':'uint8','ack_flag_cnt':'uint8',
'urg_flag_cnt':'uint8','cwe_flag_count':'uint8','ece_flag_cnt':'uint8','down_up_ratio':'uint16','pkt_size_avg':'float32','fwd_seg_size_avg':'float32','bwd_seg_size_avg':'float32','fwd_byts_b_avg':'uint8','fwd_pkts_b_avg':'uint8','fwd_blk_rate_avg':'uint8',
'bwd_byts_b_avg':'uint8','bwd_pkts_b_avg':'uint8','bwd_blk_rate_avg':'uint8','subflow_fwd_pkts':'uint32','subflow_fwd_byts':'uint32','subflow_bwd_pkts':'uint32','subflow_bwd_byts':'uint32','init_fwd_win_byts':'int32','init_bwd_win_byts':'int32','fwd_act_data_pkts':'uint32',
'fwd_seg_size_min':'uint8','active_mean':'float32','active_std':'float32','active_max':'uint32','active_min':'uint32','idle_mean':'float32','idle_std':'float32','idle_max':'uint64','idle_min':'uint64','label':'category',
'flow_id':'object','src_ip':'object','src_port':'float64','dst_ip':'object'}

In [5]:
# Load Dataset CIC-IDS2018 #
# ========================== #

m_load = time.time()
# load dataset
#dataset_cic2018 = pd.read_csv("G:\\Universitas Indonesia\\Rancangan Paper\\Dataset CSE-CIC-IDS2018-AWS\\Clean_CIC2018\\Clean_CIC2018_kali_python.csv", dtype = tipe_dataset)
dataset_cic2018 = pd.read_csv("H:\\3_hibrid_learning\\Fajar Ndolu_3 Capture Trafik_python\\benign_3003.csv")
s_load = time.time()
print ('Lama waktu load dataset : ', s_load - m_load)

Lama waktu load dataset :  0.022001266479492188


In [102]:
# mengganti nilai 'inf' menjadi 'NaN'
df_kali = df_kali.replace([np.inf, -np.inf], np.nan)

# menghitung jumlah nilai 'NaN' dalam dataset
df_kali.isnull().sum().to_numpy()

# menghapus sampel berisi nilai 'NaN'
df_kali = df_kali.dropna() 

NameError: name 'df_kali' is not defined

In [97]:
dataset_cic2018_2['flow_duration'].value_counts()

0.000000e+00    169
1.406670e+01      3
1.749706e+04      2
1.114509e+05      2
1.723409e+04      2
               ... 
4.752531e+05      1
9.662728e+05      1
6.883621e+05      1
6.175132e+05      1
2.127486e+06      1
Name: flow_duration, Length: 1117, dtype: int64

In [6]:
dataset_cic2018

,src_ip,dst_ip,src_port,dst_port,protocol,timestamp,flow_duration,flow_byts_s,flow_pkts_s,fwd_pkts_s,bwd_pkts_s,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,fwd_pkt_len_mean,fwd_pkt_len_std,bwd_pkt_len_max,bwd_pkt_len_min,bwd_pkt_len_mean,bwd_pkt_len_std,pkt_len_max,pkt_len_min,pkt_len_mean,pkt_len_std,pkt_len_var,fwd_header_len,bwd_header_len,fwd_seg_size_min,fwd_act_data_pkts,flow_iat_mean,flow_iat_max,flow_iat_min,flow_iat_std,fwd_iat_tot,fwd_iat_max,fwd_iat_min,fwd_iat_mean,fwd_iat_std,bwd_iat_tot,bwd_iat_max,bwd_iat_min,bwd_iat_mean,bwd_iat_std,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fin_flag_cnt,syn_flag_cnt,rst_flag_cnt,psh_flag_cnt,ack_flag_cnt,urg_flag_cnt,ece_flag_cnt,down_up_ratio,pkt_size_avg,init_fwd_win_byts,init_bwd_win_byts,active_max,active_min,active_mean,active_std,idle_max,idle_min,idle_mean,idle_std,fwd_byts_b_avg,fwd_pkts_b_avg,bwd_byts_b_avg,bwd_pkts_b_avg,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts,label_prediksi
0,192.168.1.2,239.255.255.250,53202,1900,17,1682311284,0,0.000000,0.000000,0.000000,0.000000,1,0,217,0,217,217,217.000000,0.000000,0,0,0.000000,0.000000,217,217,217.000000,0.00000,0.000000,8,0,8,1,0.000000e+00,0,0,0.000000e+00,0,0,0,0.000000,0.000000,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,0,0,217.000000,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,217.000000,0.000000,0,1,0,217,0,dos_attacks_slowhttptest
1,192.168.1.2,239.255.255.250,53203,1900,17,1682311284,1009238,430.027405,1.981693,1.981693,0.000000,2,0,434,0,217,217,217.000000,0.000000,0,0,0.000000,0.000000,217,217,217.000000,0.00000,0.000000,16,0,8,2,0.000000e+00,0,0,0.000000e+00,0,0,0,0.000000,0.000000,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,0,0,217.000000,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,217.000000,0.000000,0,2,0,434,0,dos_attacks_slowhttptest
2,192.168.1.2,20.195.112.105,3499,443,6,1682311286,101135,131576.585378,237.306534,108.765495,128.541039,11,13,3764,9543,1494,54,342.181818,428.804431,1506,60,734.076923,698.086887,1506,54,554.458333,621.58822,386371.914931,220,260,20,5,4.397175e+03,19556,0,7.558894e+03,83187,21625,0,8318.710327,9372.807852,82777,22149,61,6.898085e+03,9.215817e+03,0,0,0,0,0,0,0,0,0,0,0,1,554.458333,64240,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,11000.5,5.0,0.0,1.062142e+06,342.181818,734.076923,0,11,13,3764,9543,ssh_bruteforce
3,192.168.1.2,192.168.1.1,61306,53,17,1682311291,70069,3624.994355,28.543263,14.271631,14.271631,1,1,71,183,71,71,71.000000,0.000000,183,183,183.000000,0.000000,183,71,127.000000,56.00000,3136.000000,8,8,8,1,0.000000e+00,0,0,0.000000e+00,0,0,0,0.000000,0.000000,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,0,1,127.000000,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,71.000000,183.000000,0,1,1,71,183,ssh_bruteforce
4,192.168.1.2,192.168.1.1,60495,53,17,1682311291,5008104,81.867296,0.798705,0.199676,0.599029,1,3,71,339,71,71,71.000000,0.000000,134,71,113.000000,29.698485,134,71,102.500000,31.50000,992.250000,8,24,8,1,1.669368e+06,4872257,67517,2.264785e+06,0,0,0,0.000000,0.000000,4940586,4872257,68328,2.470293e+06,2.401965e+06,0,0,0,0,0,0,0,0,0,0,0,3,102.500000,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,71.000000,113.000000,0,1,3,71,339,ssh_bruteforce
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1967,192.168.1.2,13.214.78.226,8956,443,6,1682312040,17479,6922.521026,114.421835,57.210918,57.210918,1,1,55,66,55,55,55.000000,0.000000,66,66,66.000000,0.000000,66,55,60.500000,5.50000,30.250000,20,20,20,1,0.000000e+00,0,0,0.000000e+00,0,0,0,0.000000,0.

In [ ]:
dataset_cic2018_2.dtypes

In [ ]:
dataset_cic2018_2.columns.tolist()

In [7]:
dataset_cic2018['dst_port'].value_counts()

443      1566
53        159
1900       18
80         10
7587        3
1386        2
7732        2
1109        2
1554        2
1556        2
1383        2
8893        1
8854        1
1989        1
1342        1
1477        1
1384        1
1381        1
7748        1
1474        1
1511        1
1509        1
7635        1
1376        1
1967        1
1059        1
1486        1
1095        1
1357        1
1515        1
1519        1
1499        1
1954        1
8895        1
1212        1
1502        1
7698        1
1520        1
7716        1
9059        1
7715        1
1371        1
7740        1
1346        1
7766        1
7760        1
7768        1
7765        1
7763        1
1074        1
1070        1
1072        1
1385        1
1104        1
1948        1
1223        1
7745        1
7746        1
7749        1
1528        1
1522        1
8874        1
1537        1
1653        1
1651        1
7597        1
7576        1
1446        1
1480        1
1388        1
1836        1
1856  

In [65]:
data_53_2 = dataset_cic2018_2.loc[dataset_cic2018_2['dst_port'].isin([53])]
data_1900_2 = dataset_cic2018_2.loc[dataset_cic2018_2['dst_port'].isin([1900])]

In [66]:
data_53 = dataset_cic2018.loc[dataset_cic2018['dst_port'].isin([53])]
data_1900 = dataset_cic2018.loc[dataset_cic2018['dst_port'].isin([1900])]

In [68]:
data_1900['label'].value_counts()

benign                      1391
infilteration                 69
ftp_bruteforce                 6
dos_attacks_slowhttptest       3
ssh_bruteforce                 2
ddos_attack_loic_udp           2
dos_attacks_goldeneye          2
dos_attacks_slowloris          2
Name: label, dtype: int64

In [70]:
data_1900_2['label_prediksi'].value_counts()

dos_attacks_slowhttptest    80
dos_attacks_slowloris        4
ssh_bruteforce               4
Name: label_prediksi, dtype: int64

In [47]:
dataset_cic2018['label'].value_counts()

benign                      13155354
ddos_attack_hoic              686012
ddos_attacks_loic_http        576191
dos_attacks_hulk              461912
bot                           285933
ftp_bruteforce                193753
ssh_bruteforce                188337
infilteration                 183429
dos_attacks_slowhttptest      143925
dos_attacks_goldeneye          47281
dos_attacks_slowloris          16166
ddos_attack_loic_udp            1881
brute_force__web                 590
brute_force__xss                 230
sql_injection                     87
Name: label, dtype: int64

In [8]:
dataset_cic2018['label_prediksi'] = 'benign'

In [9]:
dataset_cic2018.shape

(1972, 83)

In [10]:
dataset_cic2018.to_csv('H:\\3_hibrid_learning\\benign_3003.csv', index=False)

In [24]:
dataset_cic2018_2 = pd.read_csv("G:\\Universitas Indonesia\\Rancangan Paper\\Dataset CSE-CIC-IDS2018-AWS\\pcap_test\\Pyhton\\normal_2.csv")

In [28]:
dataset_cic2018.shape

(253, 83)

In [10]:
dataset_cic2018['label_prediksi'].value_counts()

benign    100
Name: label_prediksi, dtype: int64